# Predicción del precio de alquiler de la vivienda en la ciudad de Barcelona

Descripción del dataset:

|Columna|Descripción|Key|
|--|--|--|
|id|Identificador numérico de la vivienda||
|price|Precio de mercado de la vivienda||
|currency|Moneda|Euros / Mes|
|latitude|Latitud de las coordenadas geográficas de la vivienda||
|longitude|Longitud de las coordenadas geográficas de la vivienda||
|sq_meters|Metros cuadrados de la vivienda||
|sq_meters_built|Metros cuadrados construídos de la vivienda||
|rooms|Número de habitaciones||
|bathrooms|Número de baños||
|balcony|Indicador si la vivienda tiene balcón|1, 0|
|terrace|Indicador si la vivienda tiene terraza|1, 0|
|exterior|Indicador si la vivienda tiene una orientación exterior o interior en el edificio|1, 0|
|orientation|Orientación principal de la vivienda|norte, sur, este, oeste|
|floor|Piso de la vivienda||
|rooftop|Indicador si la vivienda es un ático|1, 0|
|elevator|Indicador si el edificio de la vivienda tiene ascensor|1, 0|
|doorman|Indicador si el edificio tiene portero|1,0|
|pool|Indicador si la vivienda cuenta con piscina o derecho de uso de piscina|1,0|
|ac|Indicador si tiene aire acondicionado|1,0|
|heating|Indicador si tiene calefacción|bomba, electric, gas, individual|
|year_built|Año de construcción||
|quality|Indicador de calidad de la vivienda|2 - En buen estado|
|city|Ciudad de la vivienda||
|neighborhood|Barrio de la vivienda||
|dist_city_center|Distancia en kilómetros al centro de la ciudad||
|furniture|Indicador si la vivienda cuenta con mobiliario|1: Sin Equipar; 2: Cocina Equipada; 3: Amueblado|
|garage|Indicador si la vivienda tiene garage|1, 0|
|property_type|Tipo de vivienda||
|garden|Indicador si la vivienda cuenta con jardín|1,0|
|closest_station|Nombre de la estación de metro más cercana||
|dist_closest_station|Distancia en kilómetros a la estación de metro más cercana||
|created_at|Fecha de creación del anuncio||
|last_seen|Fecha última en la que el anuncio fue publicado en la web||



#### Carga de librerías y dataset

In [ ]:
# Library load
import os
import pandas as pd
import numpy as np

# Plotting library
import seaborn as sn
import matplotlib.pyplot as plt

# Funciones para hacer cálculo estadístico
import statsmodels.api as sm
from statsmodels.formula.api import ols

# sklearn packages
from sklearn.preprocessing import StandardScaler # Análisis de PCA
from sklearn import metrics # Calcula métricas para un modelo
from sklearn import tree # Cálculo de decision trees
from sklearn.tree import DecisionTreeClassifier # Generación de modelos de decision tree
from sklearn.ensemble import BaggingClassifier # Generación de modelos de bagging
from sklearn.ensemble import RandomForestClassifier # Generación de modelo de random forest
from sklearn.model_selection import train_test_split # Hace split entre training y testing
from sklearn.model_selection import cross_validate # trains model with cross validation
from sklearn.model_selection import GridSearchCV # Optimización de hiperparámetros para un modelo


# Models
# import lightgbm as lgb
import catboost as cat
# import xgboost as xgb

# Confusion matrix viz
from mlxtend.evaluate import confusion_matrix # Calcula la matriz de confusion 
from mlxtend.plotting import plot_confusion_matrix #plot de la matriz de confusión
from mlxtend.classifier import StackingClassifier #stacking classifier

# Shap library
import shap
# print the JS visualization code to the notebook 
shap.initjs()

# Ignoring warning messages
import warnings
warnings.filterwarnings('ignore') #ignora los errores en el notebook

### Boosting

En este punto buscamos aplicar los modelos de boosting más potentes, enfocándonos en CATBoost y aprender a optimizar los hiperparámetros para evitar el sobre-entrenamiento

- Generaión de un dataset pool para el modelo de CATBoost: ¿Cómo podemos codificar las variables categóricas?
- Optimización del modelo mediante la visualización de las curvas de aprendizaje: ¿Hay algún outlayer?, ¿Todos los modelos se comportan igual?

**Objetivos de éste punto:**

- Aprender a modelar los modelos de boosting
- Entender el funcionamiento de CATBoost

#### CATBoost

In [ ]:
!pip install ipywidget
import ipywidget

In [ ]:
# create a train/test split with catboost
d_train_cat = cat.Pool(X_train, y_train)
d_test_cat = cat.Pool(X_test, y_test)

Grid Search: https://catboost.ai/en/docs/concepts/python-reference_catboost_grid_search

In [ ]:
model = cat.CatBoostRegressor()

grid = {'learning_rate': [0.05, 0.025, 0.015, 0.01],
        'depth': [8, 10, 12]}

grid_search_result = model.grid_search(grid, 
                                       X_train, 
                                       y_train, 
                                       plot=True)

Cross validation: https://catboost.ai/en/docs/concepts/python-reference_cv

In [ ]:
# Model cross validation
params = {"iterations": n
          , "depth": n
          , "loss_function": "RMSE"
          , "learning_rate": n
          , "verbose": False}

scores = cat.cv(d_train_cat
            , params
            , fold_count = 10
            , plot = "True")

In [ ]:
model_cat = cat.CatBoostRegressor(iterations = n
                                   , learning_rate = n
                                   , loss_function = "RMSE"
                                   , random_seed = 2
                                   , depth = n)

model_cat.fit(d_train_cat
                , eval_set = d_test_cat
                , verbose_eval=150
                , early_stopping_rounds=50
                , plot = True)

In [ ]:
predictions = model_cat.predict(X_test)
metrics.r2_score(y_test, predictions)

### Model Stacking & Feature Engineering

En éste punto veremos cómo podemos hacer un modelo de modelos que nos ayude a mejorar las predicciones.

- Generación de un modelo de stacking: ¿Qué modelos podemos utilizar para nuestro modelo?
- Feature Engineering del model stacking: ¿Qué modelos debemos utilizar para nuestro meta-learner?

**Objetivos de éste punto:**

- Generar y entrenar múltiples modelos de datos y comparar los resultados obtenidos 

In [ ]:
# m1, m2, m3 son modelos generados previamente
# mf es el modelo final que queremos aplicar
sclf = StackingClassifier(classifiers=[m1, m2, m3], 
                          meta_classifier=gbt)

sclf.fit(X_train, y_train)

In [ ]:
predictions = sclf.predict(X_test)
metrics.r2_score(y_test, predictions)